In [1]:
with open('wizard_of_oz.txt' , 'r' , encoding='utf-8') as f :
    text = f.read()
print(len(text))


228251


In [2]:
print(text[:200])

LIST OF CHAPTERS


CHAPTER                                                             PAGE

  1 THE EARTHQUAKE                                                      13

  2  THE GLASS CITY           


In [3]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [4]:
string_to_int = {c:i for i,c in enumerate(chars)}
int_to_string = {i:c for i,c in enumerate(chars)}
encode = lambda s : [string_to_int[c] for c in s] 
decode = lambda x : ''.join([int_to_string[i] for i in x])

In [5]:
encode('hello')

[60, 57, 64, 64, 67]

In [6]:
decode([60, 57, 64, 64, 67])

'hello'

In [7]:
encoded_word = encode('ok computer')
encoded_word

[67, 63, 1, 55, 67, 65, 68, 73, 72, 57, 70]

In [8]:
import torch
data = torch.tensor(encode(text) , dtype=torch.long)

In [9]:
n = int(0.8*len(data))
n

182600

In [10]:
train_data = data[:n]
val_data = data[n:]

In [11]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t]
    target = y[t]
    print(f'when the context is {context} and the target is {target}')

when the context is tensor([], dtype=torch.int64) and the target is 35
when the context is tensor([79]) and the target is 32
when the context is tensor([79, 35]) and the target is 42
when the context is tensor([79, 35, 32]) and the target is 43
when the context is tensor([79, 35, 32, 42]) and the target is 1
when the context is tensor([79, 35, 32, 42, 43]) and the target is 38
when the context is tensor([79, 35, 32, 42, 43,  1]) and the target is 29
when the context is tensor([79, 35, 32, 42, 43,  1, 38]) and the target is 1


In [12]:
batch_size = 4


In [13]:
def get_batch(split):
    if split == 'train':
        data = train_data
    else:
        data = val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

x,y = get_batch('train')
x,y

(tensor([[57,  1, 72, 60, 57,  1, 54, 57],
         [57, 63, 53,  1, 65, 53, 56, 57],
         [58, 72, 57, 70,  1, 53,  1, 64],
         [70, 53, 61, 56,  1, 67, 58,  8]]),
 tensor([[ 1, 72, 60, 57,  1, 54, 57, 53],
         [63, 53,  1, 65, 53, 56, 57,  1],
         [72, 57, 70,  1, 53,  1, 64, 67],
         [53, 61, 56,  1, 67, 58,  8,  3]]))

In [14]:
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vocab_size = len(chars)
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            index_next = torch.multinomial(probs, num_samples=1) 
            index = torch.cat((index, index_next), dim=1) 
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

In [15]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



gL3adpGKeBx[-w'.2mi Jn3U!Z8uTgF3TbL"(U!rreYjuY:Z?Nd,mt(I8g,3
x]swQ5;sF4kvAxX9u]U8NfwXw'MU!-WzHca:t!OR[ngUI7p2
[ycFySs2JGu0n9EzH4F6aIgVjL4kLV26K_Pv,]ne-sGFYERj-6Bvna﻿uHMyWq_.M!]UDbDmW1Ay6vtiTx8uHwIPMUf)yox]m[Xn[b41D;Z?Q4kYGQ4dgqXHSG(kNu-N &K4qL4LUGi
0t.hbL(p5NYZudHT8tF[zdS.dh?sAPXx8&fqbdkNG_MqxARVR"othnD?
4FPs.MeN﻿DZ'Wk-SRHE
XL0ycblLpPsq9﻿wljq1JZ.eQ]MvwO1RC!rt[ :2kIwdSHD[K
D'42k"cx:7P8Vy:?mj:yrut]Rd8KrrjqtYSP8Ke8RL3SUgzJep;j:P5F1[3E
D[3x
0qVc&L"cU&I﻿1mW.)IY)]pN_M2kJ!1BNQPMk;79x'j.SHX[66eeyKB,p4m4


In [18]:

learning_rate = 3e-4
max_iters = 10000
optimiser = torch.optim.AdamW(params=model.parameters() , lr=learning_rate)
for iter in (range(max_iters)):
    if iter % eval_iters ==0:
        losses = estimate_loss()
        print(f"step : {iter} | train loss : {losses['train']} | validation loss : {losses['val']}")
    xb ,yb = get_batch('train')
    
    logits , loss = model.forward(xb ,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()
print(loss.item())

step : 0 | train loss : 4.896011829376221 | validation loss : 4.892610549926758
step : 1000 | train loss : 4.6411895751953125 | validation loss : 4.643209934234619
step : 2000 | train loss : 4.409542083740234 | validation loss : 4.421180248260498
step : 3000 | train loss : 4.214519500732422 | validation loss : 4.217954158782959
step : 4000 | train loss : 4.030304908752441 | validation loss : 4.0371198654174805
step : 5000 | train loss : 3.8509159088134766 | validation loss : 3.8579154014587402
step : 6000 | train loss : 3.710740327835083 | validation loss : 3.7031543254852295
step : 7000 | train loss : 3.538269519805908 | validation loss : 3.564873218536377
step : 8000 | train loss : 3.420499324798584 | validation loss : 3.441255569458008
step : 9000 | train loss : 3.321782112121582 | validation loss : 3.338127374649048
3.2182905673980713


In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


" tunothe to _j-r ad injuse he ng!

thtoond "I thouroavey tyowast!9whe

owioulem."DAp thetoske irie meapanderid the
he t
 dorese headared t othy WL izars
AHKKnd  cusinorsee ofe urte bey ss nce therat.
"SIGEXGaroow a w sus a te I "L4?ckn'be cr iey  isksod achaupled
s l chizshes ed os the aid sere  c sqbeqqqut sar pananoow his ad nved ve n mandie r mar athur, 're ce elekend ppant pllord a t, re.

"
Whong
" olan-s w iplcorllom Pr

basen'CAL9if the AJis?-wlecke Wembed,"YTh tth VEut wnieng,"
llieind 


In [17]:
eval_iters = 1000
@torch.no_grad()
def estimate_loss():
    out ={}
    model.eval()
    for split in ['train' , 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k]=loss.item()
        out[split] = losses.mean()
    model.train()
    return out